In [ ]:
using Pkg; Pkg.activate(".")
using Revise
import Plots: @animate, plot3d, gif, plot, @gif
import LinearAlgebra: norm, dot
import Base

include("Opt.jl")
import .Opt

In [ ]:
# pre-allocated
x0 = [0., 0.]
n_iterations = 1000

Γf = zeros(n_iterations, 3)
grad_norm = zeros(n_iterations)

function access_state(state)
    Γf[state.k,:] = [state.x..., state.f]
    grad_norm[state.k] = norm(state.g)
end

Opt.gradient_descent(x0, rosenbrock, rosenbrock_grad!;
    α = 0.001,
    n_iterations = n_iterations,
    access_state = access_state)

In [ ]:
subplot(211)
p1 = plot(1:n_iterations, Γf[:,3])
title("f(x)")
xlabel("k")

subplot(212)
p2 = plot(1:n_iterations, grad_norm)
title("|∇f(x)|")
xlabel("k")

In [ ]:
struct GradientDescentBarzilaiBorweinState
    # iteration
    k::Int64
    # iterates
    x_prev::Array{Float64, 1}
    x::Array{Float64, 1}
    # f(x)
    f::Float64
    # ∇f(x)
    g_prev::Array{Float64, 1}
    g::Array{Float64, 1}
    # stepsize α
    α::Float64

    function GradientDescentBarzilaiBorweinState(n)
        new(0, zeros(n), zeros(n), 0, zeros(n), zeros(n), 0.)
    end
end

function Base.show(io::IO, s::GradientDescentBarzilaiBorweinState)
    print("k=$(s.k) \t f(x)=$(round(s.f,sigdigits=5)) \t α=$(s.α)")
end

In [ ]:
x0 = [0., 0.]
n_iterations = 100
f = rosenbrock
grad! = rosenbrock_grad!
α₀ = 0.01
barzilaiborwein_type = 1
g_abstol = 1e-8
access_state(state) = nothing

x = copy(x0)
g = copy(x0); grad!(g, x)
u, v = zeros(size(x0)), zeros(size(x0))

s = GradientDescentBarzilaiBorweinState(size(x, 1))
grad!(s.g, x)

for k in 1:n_iterations

    s.k, s.f = k, f(x)
    @. s.x_prev = s.x
    @. s.x = x
    @. s.g_prev = s.g
    @. s.g = g

    if k == 1
        # should do line search
        s.α = α₀
    else
        u = s.x - s.x_prev
        v = s.g - s.g_prev
        s.α = (barzilaiborwein_type == 1) ?
            dot(u, v) / norm(v)^2 :
            norm(u)^2 / dot(u, v)
    end

    access_state(s)        

    x = s.x + s.α*(-s.g)
    grad!(g, x)

    if norm(s.g) <= g_abstol
        println("Terminate at k=$k: |∇f(x)| = $(norm(s.g)) <= $g_abstol")
        break
    end
end

In [ ]:
x0 = [0., 0.]
n_iterations = 1000

fs = zeros(n_iterations)
grad_norm = zeros(n_iterations)
αs = zeros(n_iterations)

function access_state(state)
    fs[state.k] = state.f
    grad_norm[state.k] = norm(state.g)
    αs[state.k] = state.α
end

Opt.gradient_descent_barzilaiborwein(x0, rosenbrock, rosenbrock_grad!;
    n_iterations = n_iterations,
    access_state = access_state)

In [ ]:
N = 50
xs = 2:N

subplot(221)
plot(xs, fs[xs])
ylabel("f(x)")
xlabel("k")

subplot(222)
plot(xs, grad_norm[xs])
ylabel("|∇f(x)|")
xlabel("k")

subplot(223)
plot(xs, αs[xs])
ylabel("α")
xlabel("k")

In [ ]:
import Plots
import Plots: @animate, plot3d, gif

step = 1
s = collect(1:step:n_iterations)
fs = fs[s]

x = range(-2;stop=2,length=100)
y = range(-1;stop=3,length=100)
z = x .+ y

plt = Plots.plot(
    range(-2;stop=2,length=100),
    range(-1;stop=3,length=100),
    (x,y) -> rosenbrock([x y]),
    st=:contourf, camera=(-30,30))

plt = Plots.plot(
    1,
    xlim = (-2, 2),
    ylim = (-1, 3),
    zlim = (0, 2500),
    title = "Descent Methods",
    marker = 3)

anim = @animate for k=1:size(s, 1)
    push!(plt, Γfs[k,1], Γfs[k,2], Γfs[k])
end

Plots.gif(anim, "tmp.gif", fps = 15)

In [ ]:
struct Bar
    A::Array{Int64, 1}
end

foo = Bar([0])
@. foo.A = 2

In [ ]:
fs